In [1]:
import optlang
import numpy as np
import sympy


from optlang import Model, Variable, Constraint, Objective

# All the (symbolic) variables are declared, with a name and optionally a lower and/or upper bound.
x1 = Variable('x1', lb=5, ub=100, type="integer")
x2 = Variable('x2', lb=0, ub=100, type="integer")
x3 = Variable('x3', lb=0, ub=100, type="integer")

# A constraint is constructed from an expression of variables and a lower and/or upper bound (lb and ub).
c1 = Constraint(x1 + x2 + x3, ub=100)
c2 = Constraint(10 * x1 + 4 * x2 + 5 * x3, ub=600)
c3 = Constraint(2 * x1 + 2 * x2 + 6 * x3, ub=300)

# An objective can be formulated
obj = Objective(10 * x1 + 6 * x2 + 4 * x3, direction='max')

# Variables, constraints and objective are combined in a Model object, which can subsequently be optimized.
model = Model(name='Simple model')
model.objective = obj
model.add([c1, c2, c3])

status = model.optimize()

print("status:", model.status)
print("objective value:", model.objective.value)
print("----------")
for var_name, var in model.variables.iteritems():
    print(var_name, "=", var.primal)

status: optimal
objective value: 732.0
----------
x1 = 33.0
x2 = 67.0
x3 = 0.0


In [2]:

def range_of_polynomial_with_bounded_vars(expr):
    expr = expr.expand()
    if not expr.is_polynomial():
        raise ValueError("The expression {} is not polynomial!".format(expr))

    min_v = 0
    max_v = 0

    # go through sum terms
    for summand in sympy.Add.make_args(expr):
        # get coefficient and product of variables
        c,vs = summand.as_coeff_Mul()
        
        min_fac = c
        max_fac = c
        # go through all factors
        for var,exp in vs.as_powers_dict().items():
            # get min and max value of factor
            val0 = var.lb**exp
            val1 = var.ub**exp
            new_vals= (min_fac*val0, min_fac*val1, max_fac*val0, max_fac*val1)
            min_fac = min(new_vals)
            max_fac = max(new_vals)
        
        min_v += min_fac
        max_v += max_fac
    
    return (min_v, max_v)
        

In [3]:
#    5 < x < 17
# -> x = 5 + x1 + 2x2 + 4x3 + 8x4, 
# -> 5 < 5 + x1 + 2x2 + 4x3 + 8x4 < 17 <=> 0 < x1 + 2x2 + 4x3 + 8x4 < 12

def binary_expansion(var: Variable):
    if var.type != "integer" and var.type != "binary":
        raise ValueError("Variable {} is not an integer, but this package can only handle integer variables!".format(var.name))
    
    # check if the variable has bounds - if so, add them as constraints, otherwise complain
    if var.lb is None:
        raise ValueError("Variable {} is does not have a defined lower bound!".format(var.name))
    if var.ub is None:
        raise ValueError("Variable {} is does not have a defined upper bound!".format(var.name))
    
    # compute value range and required number of bits to represent it
    num_bits = (int(np.ceil(np.log2(float(var.ub)-float(var.lb+1)))))
    
    # add binary variables
    sub_vars = []
    sub = var.lb
    for i in range(num_bits):
        # add the variable for the binary expansion
        sub_var = Variable("{}_{}".format(var.name, i), type="binary")
        sub_vars.append(sub_var)
        
        # add the monomial term to the expansion of the integer variable
        sub += (2**i) * sub_var
    
    # add the upper and lower bounds as a constraint on the binary expansion of the integer variable
    constraint = Constraint(sub, lb=var.lb, ub=var.ub)
    
    return sub_vars, sub, constraint

In [4]:
qubo_model = Model(name='QUBO model')
substitutions = dict()
variables = []
leq_constraints = []
zero_equations = []

# go through all variables
for var_name, var in model.variables.iteritems():
    
    # expand every non-binary variable
    if var.type != "binary":
        sub_vars, sub, constraint = binary_expansion(var)
        substitutions[var] = sub
        variables.extend(sub_vars)
        leq_constraints.append(constraint)
    else:
        variables.add(var)
    
# copy over all the explicit constraints, but substituting in the new variables and breaking them up into positive and negative constraints
for constraint in model.constraints:
    # substitute the expanded variables into the constraint expression
    expr = constraint.expression.subs(substitutions, simultaneous=False)
    
    # if both upper and lower bounds coincide, this is not an inequality constraint but rather an equation
    if constraint.ub is not None and constraint.lb is not None and constraint.ub==constraint.lb:
        # compute the value range this expression can take
        computed_lb, computed_ub = range_of_polynomial_with_bounded_vars(expr)    
    
        # check if it is even possible to satisfy this constraint, at all
        if not (computed_lb <= constraint.lb <= computed_ub):
            raise ValueError("Cannot satisfy constraint {}<={}<={} because {}<={}<={}".format(constraint.lb, constraint.expression, constraint.ub, computed_lb, expr, computed_ub))

        zero_equations.append(constraint.expression-constraint.ub)
    else:
        # otherwise, add the individual constraints to the list
        if constraint.ub is not None:
            leq_constraints.append(Constraint(expr, ub=constraint.ub))
        if constraint.lb is not None:
            leq_constraints.append(Constraint(-expr, ub=-constraint.lb))


s = 0
# introduce slack variables for all constraint equations
while len(leq_constraints) != 0:
    # take out a constraint
    constraint = leq_constraints.pop()
    
    # compute the value range this expression can take
    computed_lb, computed_ub = range_of_polynomial_with_bounded_vars(constraint.expression)    
    
    # check if it is even possible to satisfy this constraint, at all
    if computed_lb>constraint.ub:
        raise ValueError("Cannot satisfy constraint {}<={}<={} because {}>={}".format(computed_lb, constraint.expression, constraint.ub))

    # check if the constraint is trivial and can be ignored
    if computed_ub<=constraint.ub:
        print("Trivial! Contraint {} <= {} <= {}".format(constraint.expression, computed_ub, constraint.ub))
        continue

    #     constraint.expression <= constraint.ub
    # <=> constraint.expression - computed_lb + c <= constraint.ub - computed_lb + c
    # <=> constraint.expression - computed_lb + c = s;  0 <= s <= constraint.ub - computed_lb + c
    # <=> constraint.expression - computed_lb + c - s = 0; s <= constraint.ub - computed_lb + c
    
    # compute the next power of 2 for the rhs
    next_power_of_2 = (1<<int(np.ceil(np.log2(float(constraint.ub+1-computed_lb)))))-1
    
    # compute the offset needed to make the rhs a power of two
    c = next_power_of_2 - (constraint.ub - computed_lb)

    # do a binary expansion of the slack variable
    slack_vars, slack_eq, slack_constraint = binary_expansion(Variable("s_{}".format(s), type="integer", lb=0, ub=next_power_of_2))
    s+=1
    
    # add the variables
    variables.extend(slack_vars)
    
    # add the equation "constraint.expression + c - computed_lb - s = 0"; the inequality should now be trivial
    zero_equations.append(constraint.expression + c - computed_lb - slack_eq)


# copy over the objective, but with the new variables
objective = Objective((1 if model.objective.direction=="min" else -1)*model.objective.expression.subs(substitutions), direction="min")

In [5]:
for eq in zero_equations:
    print((eq**2).expand())

1.0*s_0_0**2 + 4.0*s_0_0*s_0_1 + 8.0*s_0_0*s_0_2 + 16.0*s_0_0*s_0_3 + 32.0*s_0_0*s_0_4 + 64.0*s_0_0*s_0_5 + 128.0*s_0_0*s_0_6 + 256.0*s_0_0*s_0_7 + 512.0*s_0_0*s_0_8 - 4.0*s_0_0*x1_0 - 8.0*s_0_0*x1_1 - 16.0*s_0_0*x1_2 - 32.0*s_0_0*x1_3 - 64.0*s_0_0*x1_4 - 128.0*s_0_0*x1_5 - 256.0*s_0_0*x1_6 - 4.0*s_0_0*x2_0 - 8.0*s_0_0*x2_1 - 16.0*s_0_0*x2_2 - 32.0*s_0_0*x2_3 - 64.0*s_0_0*x2_4 - 128.0*s_0_0*x2_5 - 256.0*s_0_0*x2_6 - 12.0*s_0_0*x3_0 - 24.0*s_0_0*x3_1 - 48.0*s_0_0*x3_2 - 96.0*s_0_0*x3_3 - 192.0*s_0_0*x3_4 - 384.0*s_0_0*x3_5 - 768.0*s_0_0*x3_6 - 442.0*s_0_0 + 4.0*s_0_1**2 + 16.0*s_0_1*s_0_2 + 32.0*s_0_1*s_0_3 + 64.0*s_0_1*s_0_4 + 128.0*s_0_1*s_0_5 + 256.0*s_0_1*s_0_6 + 512.0*s_0_1*s_0_7 + 1024.0*s_0_1*s_0_8 - 8.0*s_0_1*x1_0 - 16.0*s_0_1*x1_1 - 32.0*s_0_1*x1_2 - 64.0*s_0_1*x1_3 - 128.0*s_0_1*x1_4 - 256.0*s_0_1*x1_5 - 512.0*s_0_1*x1_6 - 8.0*s_0_1*x2_0 - 16.0*s_0_1*x2_1 - 32.0*s_0_1*x2_2 - 64.0*s_0_1*x2_3 - 128.0*s_0_1*x2_4 - 256.0*s_0_1*x2_5 - 512.0*s_0_1*x2_6 - 24.0*s_0_1*x3_0 - 48.0*s_0_1

In [60]:
Q = sympy.zeros(len(variables),len(variables))

# make matrix with coefficients on the diagonal
Q += sympy.diag(*sympy.Matrix([objective.expression]).jacobian(variables))
penalty=10
zero_equations_mat = sympy.Matrix(zero_equations)
variables_mat = sympy.Matrix(variables)
# get the coefficients of each equation
zero_equations_Jac = zero_equations_mat.jacobian(variables)

# Add quadratic terms from the constraints:
Q += penalty*zero_equations_Jac.T*zero_equations_Jac
# Q += penalty*sympy.Add(*(sympy.hessian(eq**2, variables)  for eq in zero_equations))

# Get the constant term (= the rest)
cs = zero_equations_mat-zero_equations_Jac*variables_mat

# Add the linear terms from the constraints:
Q += sympy.diag(*(penalty*2*(cs.T*zero_equations_Jac)))
print(Q)

Matrix([[105770.000000000, 2120.00000000000, 4240.00000000000, 8480.00000000000, 16960.0000000000, 33920.0000000000, 67840.0000000000, 450.000000000000, 900.000000000000, 1800.00000000000, 3600.00000000000, 7200.00000000000, 14400.0000000000, 28800.0000000000, 630.000000000000, 1260.00000000000, 2520.00000000000, 5040.00000000000, 10080.0000000000, 20160.0000000000, 40320.0000000000, -20.0000000000000, -40.0000000000000, -80.0000000000000, -160.000000000000, -320.000000000000, -640.000000000000, -1280.00000000000, -2560.00000000000, -5120.00000000000, -100.000000000000, -200.000000000000, -400.000000000000, -800.000000000000, -1600.00000000000, -3200.00000000000, -6400.00000000000, -12800.0000000000, -25600.0000000000, -51200.0000000000, -10.0000000000000, -20.0000000000000, -40.0000000000000, -80.0000000000000, -160.000000000000, -320.000000000000, -640.000000000000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -10, -20, -40, -80, -160, -320, -640], [2120.00000000000, 213660.000000000, 8

In [61]:
var_a = sympy.Array(variables, (len(variables),1))
(var_a.transpose() @ Q @ var_a)[0].expand()

-4410.0*s_0_0**2 + 40*s_0_0*s_0_1 + 80*s_0_0*s_0_2 + 160*s_0_0*s_0_3 + 320*s_0_0*s_0_4 + 640*s_0_0*s_0_5 + 1280*s_0_0*s_0_6 + 2560*s_0_0*s_0_7 + 5120*s_0_0*s_0_8 - 40.0*s_0_0*x1_0 - 80.0*s_0_0*x1_1 - 160.0*s_0_0*x1_2 - 320.0*s_0_0*x1_3 - 640.0*s_0_0*x1_4 - 1280.0*s_0_0*x1_5 - 2560.0*s_0_0*x1_6 - 40.0*s_0_0*x2_0 - 80.0*s_0_0*x2_1 - 160.0*s_0_0*x2_2 - 320.0*s_0_0*x2_3 - 640.0*s_0_0*x2_4 - 1280.0*s_0_0*x2_5 - 2560.0*s_0_0*x2_6 - 120.0*s_0_0*x3_0 - 240.0*s_0_0*x3_1 - 480.0*s_0_0*x3_2 - 960.0*s_0_0*x3_3 - 1920.0*s_0_0*x3_4 - 3840.0*s_0_0*x3_5 - 7680.0*s_0_0*x3_6 - 8800.0*s_0_1**2 + 160*s_0_1*s_0_2 + 320*s_0_1*s_0_3 + 640*s_0_1*s_0_4 + 1280*s_0_1*s_0_5 + 2560*s_0_1*s_0_6 + 5120*s_0_1*s_0_7 + 10240*s_0_1*s_0_8 - 80.0*s_0_1*x1_0 - 160.0*s_0_1*x1_1 - 320.0*s_0_1*x1_2 - 640.0*s_0_1*x1_3 - 1280.0*s_0_1*x1_4 - 2560.0*s_0_1*x1_5 - 5120.0*s_0_1*x1_6 - 80.0*s_0_1*x2_0 - 160.0*s_0_1*x2_1 - 320.0*s_0_1*x2_2 - 640.0*s_0_1*x2_3 - 1280.0*s_0_1*x2_4 - 2560.0*s_0_1*x2_5 - 5120.0*s_0_1*x2_6 - 240.0*s_0_1*x3_

In [62]:
(objective.expression + penalty*(sympy.Add(*(eq**2 for eq in zero_equations)))).expand()

10.0*s_0_0**2 + 40.0*s_0_0*s_0_1 + 80.0*s_0_0*s_0_2 + 160.0*s_0_0*s_0_3 + 320.0*s_0_0*s_0_4 + 640.0*s_0_0*s_0_5 + 1280.0*s_0_0*s_0_6 + 2560.0*s_0_0*s_0_7 + 5120.0*s_0_0*s_0_8 - 40.0*s_0_0*x1_0 - 80.0*s_0_0*x1_1 - 160.0*s_0_0*x1_2 - 320.0*s_0_0*x1_3 - 640.0*s_0_0*x1_4 - 1280.0*s_0_0*x1_5 - 2560.0*s_0_0*x1_6 - 40.0*s_0_0*x2_0 - 80.0*s_0_0*x2_1 - 160.0*s_0_0*x2_2 - 320.0*s_0_0*x2_3 - 640.0*s_0_0*x2_4 - 1280.0*s_0_0*x2_5 - 2560.0*s_0_0*x2_6 - 120.0*s_0_0*x3_0 - 240.0*s_0_0*x3_1 - 480.0*s_0_0*x3_2 - 960.0*s_0_0*x3_3 - 1920.0*s_0_0*x3_4 - 3840.0*s_0_0*x3_5 - 7680.0*s_0_0*x3_6 - 4420.0*s_0_0 + 40.0*s_0_1**2 + 160.0*s_0_1*s_0_2 + 320.0*s_0_1*s_0_3 + 640.0*s_0_1*s_0_4 + 1280.0*s_0_1*s_0_5 + 2560.0*s_0_1*s_0_6 + 5120.0*s_0_1*s_0_7 + 10240.0*s_0_1*s_0_8 - 80.0*s_0_1*x1_0 - 160.0*s_0_1*x1_1 - 320.0*s_0_1*x1_2 - 640.0*s_0_1*x1_3 - 1280.0*s_0_1*x1_4 - 2560.0*s_0_1*x1_5 - 5120.0*s_0_1*x1_6 - 80.0*s_0_1*x2_0 - 160.0*s_0_1*x2_1 - 320.0*s_0_1*x2_2 - 640.0*s_0_1*x2_3 - 1280.0*s_0_1*x2_4 - 2560.0*s_0_1*x2

In [63]:
Q

Matrix([
[105770.0,    2120.0,    4240.0,    8480.0,   16960.0,    33920.0,    67840.0,    450.0,    900.0,   1800.0,    3600.0,    7200.0,   14400.0,    28800.0,    630.0,   1260.0,    2520.0,    5040.0,   10080.0,   20160.0,    40320.0,   -20.0,   -40.0,    -80.0,   -160.0,   -320.0,    -640.0,   -1280.0,   -2560.0,   -5120.0,  -100.0,   -200.0,   -400.0,   -800.0,   -1600.0,   -3200.0,   -6400.0,   -12800.0,   -25600.0,   -51200.0,  -10.0,   -20.0,   -40.0,   -80.0,   -160.0,   -320.0,   -640.0,    0,     0,     0,     0,      0,      0,      0,    0,     0,     0,     0,      0,      0,      0,    -10,     -20,     -40,     -80,    -160,     -320,   -640],
[  2120.0,  213660.0,    8480.0,   16960.0,   33920.0,    67840.0,   135680.0,    900.0,   1800.0,   3600.0,    7200.0,   14400.0,   28800.0,    57600.0,   1260.0,   2520.0,    5040.0,   10080.0,   20160.0,   40320.0,    80640.0,   -40.0,   -80.0,   -160.0,   -320.0,   -640.0,   -1280.0,   -2560.0,   -5120.0,  -10240.0,  -200.0, 

In [64]:
def substitute_integer_to_binary(values: dict, substitutions: dict):
    substitutions_mat = sympy.Matrix(list(substitutions.values()))
    variables_mat = sympy.Matrix(variables)
    M = substitutions_mat.jacobian(variables)
    b = substitutions_mat-M*variables_mat
    var_index = dict(zip(substitutions.keys(),range(len(substitutions))))
    ret = sympy.zeros(M.shape[1],1)
    for var,val in values.items():
        i = var_index[var]
        grad = M[i,:]
        bb = b[i]
        ret += sympy.Matrix([[1 if g != 0 and (val-bb) % (2*g) >= g else 0] for g in grad])
    return ret

def substitute_binary_to_integer(values, substitutions):
    # substitutions_mat = sympy.Matrix(list(substitutions.values()))
    # variables_mat = sympy.Matrix(variables)
    # M = substitutions_mat.jacobian(variables)
    # b = substitutions_mat-M*variables_mat
    s = dict(zip(variables, values))
    return dict( (k,eq.subs(s)) for k,eq in substitutions.items())

In [65]:
binary = substitute_integer_to_binary({x1:7, x2:1, x3: 31}, substitutions)
binary.T

Matrix([[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [66]:
substitute_binary_to_integer(binary, substitutions)

{5 <= x1 <= 100: 7, 0 <= x2 <= 100: 1, 0 <= x3 <= 100: 31}

In [79]:
import neal
import dimod

bqm=dimod.BinaryQuadraticModel(np.array(Q,dtype=int), "BINARY")
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, beta_schedule_type="geometric", num_reads=1000, num_sweeps=1000)
sampleset.relabel_variables(dict(enumerate(variables)))
#decoded_samples = model.decode_sampleset(sampleset)
#best_sample = min(decoded_samples, key=lambda x: x.energy)
#pprint(best_sample.sample)
best=sampleset.lowest().first.sample

sol_bin = np.array([best[v] for v in variables])
sol_bin

array([0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0], dtype=int8)

In [80]:
sol_int=substitute_binary_to_integer(sympy.Matrix(sol_bin), substitutions)
sol_int

{5 <= x1 <= 100: 35, 0 <= x2 <= 100: 62, 0 <= x3 <= 100: 0}

In [81]:
model.objective.expression.subs(sol_int)

722.000000000000

In [82]:
objective.expression.subs(dict(zip(variables, sol_bin)))

-722.000000000000